In [1]:
import torch
normal_path = "/workspace/MILForVideos/extractors/i3d/datasets/UCSD_Anomaly_Dataset_v1p2_ped1_test_normal.pt"
video = torch.load(normal_path)[0]
video.features

tensor([[[0.0389, 0.0569, 0.5105,  ..., 0.0000, 0.0114, 0.3346]],

        [[0.0679, 0.0368, 0.4137,  ..., 0.0000, 0.0000, 0.1843]],

        [[0.1568, 0.0480, 0.3986,  ..., 0.0000, 0.0673, 0.2804]],

        ...,

        [[0.0726, 0.0715, 0.3035,  ..., 0.0000, 0.0000, 0.1266]],

        [[0.0944, 0.0821, 0.3256,  ..., 0.0000, 0.0000, 0.1191]],

        [[0.0590, 0.0441, 0.2845,  ..., 0.0000, 0.0000, 0.0921]]])

In [2]:
video.features.size()

torch.Size([13, 1, 1024])

In [3]:
video.path_list[:1]

[['/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/001.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/002.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/003.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/004.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/005.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/006.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/007.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/008.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/009.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/010.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/011.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/012.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/013.tif',
  '/datasets/UCSD_Anomaly_Dataset_v1p2/UCSDped1/Test/Test001/014.tif',
  '/da

In [4]:
from PIL import Image
from extractor import MyNet, img2tensor

idx = 5

net = MyNet()
net = net.eval().cuda()
with torch.no_grad():
    images = img2tensor(video.path_list[idx]).cuda()
    feature = net(images)[0, 0].cpu()
    display(video.features[idx, 0, ~(video.features[idx, 0] == feature)].size())

torch.Size([0])

In [5]:
import sys
import torch
import torch.nn as nn
from i3d import InceptionI3d
import argparse
import pandas as pd
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from milforvideo.video import Extractor


pathlist = "/datasets/UCSD_Anomaly_Dataset_v1p2/ped1_test_normal.txt"

# Get the image path and label from a input file
with open(pathlist) as f:
    grp_path_and_label = [
        row.split(" ")
        for row in f.read().split("\n")
        if row
    ]
    df = pd.DataFrame({
        "grp": [int(grp) for grp, _, _ in grp_path_and_label],
        "path": [path for _, path, _ in grp_path_and_label],
        "label": [int(label) for _, _, label in grp_path_and_label],
    })

net = MyNet()
outputs = []
grp, df_grp = list(df.groupby("grp"))[0]
extractor = Extractor(
    df_grp["path"].tolist(), 
    df_grp["label"].tolist(), 
    net, img2tensor, 
    F=16,
    aggregate=lambda labels: [max(labels)],
    cuda=True)
new_video = extractor.extract()
new_video.features

tensor([[[0.0389, 0.0569, 0.5105,  ..., 0.0000, 0.0114, 0.3346]],

        [[0.0679, 0.0368, 0.4137,  ..., 0.0000, 0.0000, 0.1843]],

        [[0.1568, 0.0480, 0.3986,  ..., 0.0000, 0.0673, 0.2804]],

        ...,

        [[0.0726, 0.0715, 0.3035,  ..., 0.0000, 0.0000, 0.1266]],

        [[0.0944, 0.0821, 0.3256,  ..., 0.0000, 0.0000, 0.1191]],

        [[0.0590, 0.0441, 0.2845,  ..., 0.0000, 0.0000, 0.0921]]])

In [6]:
display(video.features[0, 0, video.features[0, 0] == new_video.features[0, 0]])
display(video.features[0, 0, ~(video.features[0, 0] == new_video.features[0, 0])].size())

tensor([0.0389, 0.0569, 0.5105,  ..., 0.0000, 0.0114, 0.3346])

torch.Size([0])

In [7]:
video.features[0, 0]

tensor([0.0389, 0.0569, 0.5105,  ..., 0.0000, 0.0114, 0.3346])

In [8]:
new_video.features[0, 0]

tensor([0.0389, 0.0569, 0.5105,  ..., 0.0000, 0.0114, 0.3346])